### Library Import

In [1]:
from fileinput import filename
import os
import random
import json
import numpy as np
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import f1_score
from utils import settings as settings
import pandas as pd
from evaluation.metrics import accuracy
from evaluation.conf_matrix import create_conf_matrix
from loader.Preprocessor import Preprocessor
from models.RainbowModel import RainbowModel
from models.ResNetModel import ResNetModel
from models.ResNetModel_Multimodal import ResNetModelMultimodal
from models.LeanderDeepConvLSTM import LeanderDeepConvLSTM
from utils.filter_activities import filter_activities
from utils.folder_operations import new_saved_experiment_folder
from utils.DataConfig import Sonar22CategoriesConfig, OpportunityConfig, SonarConfig, LabPoseConfig
from tensorflow.keras.layers import (Dense)
from utils.Recording import Recording
import matplotlib.pyplot as plt
import utils.DataConfig
from pose_sequence_loader import *
from multiprocessing import Pool

### Init Settings & Hyper Parameters

In [2]:
data_config = LabPoseConfig(dataset_path='/dhc/groups/bp2021ba1/data/lab_data_filtered_without_null')#OpportunityConfig(dataset_path='/dhc/groups/bp2021ba1/data/opportunity-dataset')
#data_config = SonarConfig(dataset_path='/dhc/groups/bp2021ba1/data/lab_data')#OpportunityConfig(dataset_path='/dhc/groups/bp2021ba1/data/opportunity-dataset')
settings.init(data_config)
random.seed(1678978086101)

k_fold_splits = 5
numEpochs = 10
window_size = 600

# LOAD DATA
recordings = settings.DATA_CONFIG.load_dataset()#limit=3)
recordings = Preprocessor().our_preprocess(recordings)


Loading recording 0_orhan_1652085453257.csv, 1 / 51
3
Loading recording 1_orhan_1651673896419.csv, 2 / 51
3
Loading recording 2_alex_1652878074983.csv, 3 / 51
3
Loading recording 3_daniel_1652429334344.csv, 4 / 51
6
Loading recording 4_lucas_1651566794296.csv, 5 / 51
1
Loading recording 5_orhan_1653467107120.csv, 6 / 51
3
Loading recording 6_orhan_1652086187708.csv, 7 / 51
1
Loading recording 7_marco_1651225541980.csv, 8 / 51
3
Loading recording 8_felix_1651152283882.csv, 9 / 51
5
Loading recording 9_orhan_1653481329073.csv, 10 / 51
8
Loading recording 10_valentin_1652869027863.csv, 11 / 51
3
Loading recording 11_orhan_1651674660480.csv, 12 / 51
6
Loading recording 12_orhan_1653480087545.csv, 13 / 51
3
Loading recording 13_orhan_1651674075450.csv, 14 / 51
10
Loading recording 14_franz_1653471714380.csv, 15 / 51
1
Loading recording 15_felix_1651154555588.csv, 16 / 51
1
Loading recording 16_kirill_1651147138832.csv, 17 / 51
1
Loading recording 17_orhan_1653465446990.csv, 18 / 51
8
Loadin

### Append Pose Estimation Sequences to Recordings

In [4]:


def process_recording(recording_with_index):
    i = recording_with_index[0]
    recording = recording_with_index[1]
    pose_frame = get_poseframe(recording, "/dhc/groups/bp2021ba1/data/lab_data")
    
    print(f"Pose Frame added to Recording {i+1}/{len(recordings)}  ")
    return pose_frame

def append_pose_frames(recordings, useMultiprocessing = True):
    if useMultiprocessing:
        pool = Pool()
        pose_frames = pool.map(process_recording, list(enumerate(recordings)), 1)
        pool.close()
        pool.join()
    else:
        pose_frames = []
        for i, k in list(enumerate(recordings)):
            pose_frames.append(process_recording((i,k)))

    for i, pose_frame in enumerate(pose_frames):
        recordings[i].pose_frame = pose_frame

append_pose_frames(recordings)

initialLength = len(recordings)
recordings = list(filter(
    lambda recording: not recording.pose_frame.empty, 
    recordings
))
print(f"Filtered out {initialLength - len(recordings)} Recordings (!)")

print("==> APPENDING POSE FRAMES DONE")

Recording 1652429334344: Metadata has less than 2 Activities. Returning empty pose frame (!)
Pose Frame added to Recording 4/51  
Pose Frame added to Recording 2/51  
Pose Frame added to Recording 7/51  
Pose Frame added to Recording 9/51  
Pose Frame added to Recording 11/51  
Recording 1651674660480: Activity TimeStamps of Frame & Metadata significantly shifted. Returning empty pose frame (!)
Pose Frame added to Recording 12/51  
Recording 1651674075450: Metadata has less than 2 Activities. Returning empty pose frame (!)
Pose Frame added to Recording 14/51  
Pose Frame added to Recording 5/51  
Pose Frame added to Recording 10/51  
Pose Frame added to Recording 6/51  
Pose Frame added to Recording 1/51  
Recording 1653480087545: Activity TimeStamps of Frame & Metadata significantly shifted. Returning empty pose frame (!)
Pose Frame added to Recording 13/51  
Pose Frame added to Recording 3/51  
Pose Frame added to Recording 21/51  
Recording 1652428200498: Activity TimeStamps of Fram

### Helper for Training & Evalutation

In [ ]:
# CONFIG TRAINING
n_sensor_features = recordings[0].sensor_frame.shape[1]
n_pose_features = recordings[0].pose_frame.shape[1]
print(f"Sensor features: {n_sensor_features},  Pose features: {n_pose_features}")
n_outputs = settings.DATA_CONFIG.n_activities()

# Create Folder, save model export and evaluations there
experiment_folder_path = new_saved_experiment_folder(
    "multiModal_alex"
)

class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

def evaluate(y_test_pred: np.ndarray, y_test_true: np.ndarray, confusionMatrixFileName=None, confusionMatrixTitle="") -> tuple[float, float,float, np.ndarray]:
    acc = accuracy(y_test_pred, y_test_true)
    if confusionMatrixFileName:
        create_conf_matrix(
            experiment_folder_path, y_test_pred, y_test_true, 
            file_name = confusionMatrixFileName, title=confusionMatrixTitle+", acc:"+str(int(acc*10000)/100)+"%"
            # label_mapping = {value: key for key, value in data_config.category_labels.items()}
    ) 
    f1_macro = f1_score(np.argmax(y_test_true, axis=1), np.argmax(y_test_pred, axis=1), average="macro")   
    f1_weighted = f1_score(np.argmax(y_test_true, axis=1), np.argmax(y_test_pred, axis=1), average="weighted")    
    return acc, f1_macro, f1_weighted, y_test_true

def save_prediction(y_test_pred: np.ndarray, y_test_true: np.ndarray, acc, f1_macro, f1_weighted, file_name: str, mode: str):
    prefix = f"{mode}, WS {window_size},"
    experiment_folder_root =  os.path.join(*experiment_folder_path.split("/")[:-1])  
    experiment_folder_end = experiment_folder_path.split("/")[-1]
    experiment_folder = os.path.join(experiment_folder_root, prefix + experiment_folder_end)
    with open(os.path.join(experiment_folder_path , f"{file_name}.json"), "w+") as file:
        json_dict = {
            'Predicted Labels': y_test_pred,
            'Actual Lables': y_test_true,
            'Accuracy': acc,
            'F1 Macro': f1_macro,
            'F1 Weighted':f1_weighted
        }
        
        json.dump(json_dict, file, cls=NumpyEncoder)


def instanciateModel(use_sensor_frame = True, use_pose_frame = False) -> ResNetModelMultimodal:
    n_features = 0
    n_features += n_sensor_features if (use_sensor_frame) else 0
    n_features += n_pose_features if (use_pose_frame) else 0

    #return ResNetModelMultimodal(
    return LeanderDeepConvLSTM(
        n_epochs=numEpochs,
        window_size=window_size,
        n_features=n_features,
        n_outputs=n_outputs,
        learning_rate=0.001,
        batch_size=64,
        use_sensor_frame=use_sensor_frame,
        use_pose_frame=use_pose_frame,
        verbose = True
    )

### Leave-Recording-Out K-Fold Experiment

In [ ]:

def leave_recording_out_run():
    print("############################### LEAVE RECORDING OUT ##########################")
    print("##############################################################################")
    # TRAIN AND PREDICT
    multiModals = [(True, False), (True, True), (False, True)]
    for modality_index, (use_sensor_frame, use_pose_frame) in enumerate(multiModals):
        print(f"\n\n==== Sensor Features: {use_sensor_frame}   Pose Features: {use_pose_frame} ====")
        model = instanciateModel(use_sensor_frame=use_sensor_frame, use_pose_frame=use_pose_frame)
        model.n_epochs = numEpochs
        model.model.save_weights("ckpt")

        confusion_y_test_pred = None
        confusion_y_test_actual = None
        avg_acc = avg_f1_macro = avg_f1_weighted = divider = 0
        k_fold = KFold(n_splits=k_fold_splits, random_state=42, shuffle=True)
        for k_fold_index, (train_indices, test_indices) in enumerate(k_fold.split(recordings)):
            model.model.load_weights("ckpt")
            
            recordingsTrain = [recordings[ind] for ind in train_indices.astype(int)]
            recordingsTest = [recordings[ind] for ind in test_indices.astype(int)]

            x_train, y_train = model.windowize_convert(recordingsTrain)
            model.fit(x_train=x_train, y_train=y_train)

            x_test, y_test_actual = model.windowize_convert(recordingsTest)
            y_test_pred = model.predict(x_test)

            confusion_y_test_pred = y_test_pred if confusion_y_test_pred is None else np.append(confusion_y_test_pred, y_test_pred, axis=0)
            confusion_y_test_actual = y_test_actual if confusion_y_test_actual is None else np.append(confusion_y_test_actual, y_test_actual, axis=0)

            acc, f1_macro, f1_weighted, _ = evaluate(
                y_test_pred, 
                y_test_actual
            )

            weight = len(confusion_y_test_actual)
            avg_acc += acc * weight
            avg_f1_macro += f1_macro * weight
            avg_f1_weighted += f1_weighted * weight
            divider += weight

            print(f"=> K-Fold {k_fold_index+1}/{k_fold_splits}: acc: {acc}  f1 macro: {f1_macro}  f1 weighted: {f1_weighted}\n")

        avg_acc = avg_acc / divider
        avg_f1_macro = avg_f1_macro / divider
        avg_f1_weighted = avg_f1_weighted / divider

        save_prediction(
                confusion_y_test_pred, confusion_y_test_actual, 
                avg_acc, avg_f1_macro, avg_f1_weighted,
                f"Data {'SENSOR' if use_sensor_frame else 'x'} - {'POSE' if use_pose_frame else 'x'}",
                mode = "Rec"
            )
        _, _, _, _ = evaluate(
                confusion_y_test_pred, confusion_y_test_actual, 
                confusionMatrixFileName=f"Confusion {'SENSOR' if use_sensor_frame else 'x'} - {'POSE' if use_pose_frame else 'x'}"
            )

        print(f"\n\n==== Results ====")
        print(f"Accuracy: {avg_acc}")
        print(f"F1 Macro: {avg_f1_macro}")
        print(f"F1 Weighted: {avg_f1_weighted}")

### Leave-Window-Out K-Fold Experiment

In [ ]:
def leave_window_out_run():
    print("############################### LEAVE WINDOW OUT ##########################")
    print("##############################################################################")
    # TRAIN AND PREDICT
    multiModals = [(True, False), (True, True), (False, True)]
    for modality_index, (use_sensor_frame, use_pose_frame) in enumerate(multiModals):
        print(f"\n\n==== Sensor Features: {use_sensor_frame}   Pose Features: {use_pose_frame} ====")
        model = instanciateModel(use_sensor_frame=use_sensor_frame, use_pose_frame=use_pose_frame)
        model.n_epochs = numEpochs
        model.model.save_weights("ckpt")

        windows = model.windowize(recordings)

        confusion_y_test_pred = None
        confusion_y_test_actual = None
        avg_acc = avg_f1_macro = avg_f1_weighted = divider = 0
        k_fold = KFold(n_splits=k_fold_splits, random_state=42, shuffle=True)
        for k_fold_index, (train_indices, test_indices) in enumerate(k_fold.split(windows)):
            model.model.load_weights("ckpt")
            
            windows_train = [windows[ind] for ind in train_indices.astype(int)]
            windows_test = [windows[ind] for ind in test_indices.astype(int)]

            x_train, y_train = model.convert(windows_train)
            model.fit(x_train=x_train, y_train=y_train)

            x_test, y_test_actual = model.convert(windows_test)
            y_test_pred = model.predict(x_test)

            confusion_y_test_pred = y_test_pred if confusion_y_test_pred is None else np.append(confusion_y_test_pred, y_test_pred, axis=0)
            confusion_y_test_actual = y_test_actual if confusion_y_test_actual is None else np.append(confusion_y_test_actual, y_test_actual, axis=0)

            acc, f1_macro, f1_weighted, _ = evaluate(
                y_test_pred, 
                y_test_actual
            )

            weight = len(confusion_y_test_actual)
            avg_acc += acc * weight
            avg_f1_macro += f1_macro * weight
            avg_f1_weighted += f1_weighted * weight
            divider += weight

            print(f"=> K-Fold {k_fold_index+1}/{k_fold_splits}: acc: {acc}  f1 macro: {f1_macro}  f1 weighted: {f1_weighted}\n")

        avg_acc = avg_acc / divider
        avg_f1_macro = avg_f1_macro / divider
        avg_f1_weighted = avg_f1_weighted / divider

        save_prediction(
                confusion_y_test_pred, confusion_y_test_actual, 
                avg_acc, avg_f1_macro, avg_f1_weighted,
                f"Data {'SENSOR' if use_sensor_frame else 'x'} - {'POSE' if use_pose_frame else 'x'}",
                mode = "Win"
            )
        _, _, _, _ = evaluate(
                confusion_y_test_pred, confusion_y_test_actual, 
                confusionMatrixFileName=f"Confusion {'SENSOR' if use_sensor_frame else 'x'} - {'POSE' if use_pose_frame else 'x'}"
            )

        print(f"\n\n==== Results ====")
        print(f"Accuracy: {avg_acc}")
        print(f"F1 Macro: {avg_f1_macro}")
        print(f"F1 Weighted: {avg_f1_weighted}")

### Leave-Subject-Out Experiment

In [ ]:
def leave_subject_out_run():
    print("############################### LEAVE SUBJECT OUT ##########################")
    print("#############################################################################")

    subjects = set([recording.subject for recording in recordings])
    # TRAIN AND PREDICT
    multiModals = [(True, False), (True, True), (False, True)]
    for modality_index, (use_sensor_frame, use_pose_frame) in enumerate(multiModals):
        print(f"\n\n==== Sensor Features: {use_sensor_frame}   Pose Features: {use_pose_frame} ====")
        model = instanciateModel(use_sensor_frame=use_sensor_frame, use_pose_frame=use_pose_frame)
        model.n_epochs = numEpochs
        model.model.save_weights("ckpt")

        confusion_y_test_pred = None
        confusion_y_test_actual = None
        avg_acc = avg_f1_macro = avg_f1_weighted = divider = 0
        for subjectIndex, leaveOutSubject in enumerate(subjects):
            model.model.load_weights("ckpt")

            recordings_train = [recording for recording in recordings if recording.subject != leaveOutSubject]
            recordings_test = [recording for recording in recordings if recording.subject == leaveOutSubject]

            x_train, y_train = model.windowize_convert(recordings_train)
            model.fit(x_train=x_train, y_train=y_train)

            x_test, y_test_actual = model.windowize_convert(recordings_test)
            y_test_pred = model.predict(x_test)

            confusion_y_test_pred = y_test_pred if confusion_y_test_pred is None else np.append(confusion_y_test_pred, y_test_pred, axis=0)
            confusion_y_test_actual = y_test_actual if confusion_y_test_actual is None else np.append(confusion_y_test_actual, y_test_actual, axis=0)

            acc, f1_macro, f1_weighted, _ = evaluate(
                y_test_pred, 
                y_test_actual
            )

            weight = len(confusion_y_test_actual)
            avg_acc += acc * weight
            avg_f1_macro += f1_macro * weight
            avg_f1_weighted += f1_weighted * weight
            divider += weight

            print(f"=> Subject {subjectIndex+1}/{len(subjects)}: acc: {acc}  f1 macro: {f1_macro}  f1 weighted: {f1_weighted}\n")

        avg_acc = avg_acc / divider
        avg_f1_macro = avg_f1_macro / divider
        avg_f1_weighted = avg_f1_weighted / divider

        save_prediction(
                confusion_y_test_pred, confusion_y_test_actual, 
                avg_acc, avg_f1_macro, avg_f1_weighted,
                f"Data {'SENSOR' if use_sensor_frame else 'x'} - {'POSE' if use_pose_frame else 'x'}",
                mode = "Subj"
            )
        _, _, _, _ = evaluate(
                confusion_y_test_pred, confusion_y_test_actual, 
                confusionMatrixFileName=f"Confusion {'SENSOR' if use_sensor_frame else 'x'} - {'POSE' if use_pose_frame else 'x'}"
            )

        print(f"\n\n==== Results ====")
        print(f"Accuracy: {avg_acc}")
        print(f"F1 Macro: {avg_f1_macro}")
        print(f"F1 Weighted: {avg_f1_weighted}")

### Run Exeriments

In [ ]:
leave_recording_out_run()
leave_window_out_run()
leave_subject_out_run()

In [10]:
recordings = list(filter(
    lambda recording: not recording.pose_frame.empty, 
    recordings
))

In [11]:
from numpy import size

print(len(recordings))

for r in recordings:
    print(len(r.time_frame),len(r.activities),len(r.sensor_frame),len(r.pose_frame))


41
33997 33997 33997 33997
8788 8788 8788 8788
36641 36641 36641 36641
35555 35555 35555 35555
36521 36521 36521 36521
16939 16939 16939 16939
74915 74915 74915 74915
23015 23015 23015 23015
23883 23883 23883 23883
7483 7483 7483 7483
58993 58993 58993 58993
21139 21139 21139 21139
35706 35706 35706 35706
25052 25052 25052 25052
25356 25356 25356 25356
2752 2752 2752 2752
25852 25852 25852 25852
18937 18937 18937 18937
31664 31664 31664 31664
41591 41591 41591 41591
51041 51041 51041 51041
49105 49105 49105 49105
21424 21424 21424 21424
26687 26687 26687 26687
50123 50123 50123 50123
11742 11742 11742 11742
46128 46128 46128 46128
51715 51715 51715 51715
34589 34589 34589 34589
16358 16358 16358 16358
39763 39763 39763 39763
38520 38520 38520 38520
10788 10788 10788 10788
48943 48943 48943 48943
36946 36946 36946 36946
10037 10037 10037 10037
31142 31142 31142 31142
43414 43414 43414 43414
45219 45219 45219 45219
57752 57752 57752 57752
29118 29118 29118 29118


In [18]:

for recording in recordings:
    complete_frame = pd.concat([recording.time_frame, recording.activities], axis=1)
    complete_frame = pd.concat([complete_frame, recording.sensor_frame], axis=1)
    complete_frame = pd.concat([complete_frame, recording.pose_frame], axis=1)

    filename = f"{recording.recording_index} - {recording.subject} - {recording.recording_folder}"  
    complete_frame.to_csv(f"/dhc/groups/bp2021ba1/alex/UnicornML/src/skeleton_imu_csv_data/{filename}.csv")

SyntaxError: invalid syntax (1089694856.py, line 1)